#LAB 2: Signal to Noise Ratio, Quantization/
**ECEN432: Data Conversion Systems and Circuits**

In [ ]:
'''1. SIGNAL TO NOISE RATIO (40)
Generate a tone with frequency 2 MHz and amplitude 1 V. Sample the
tone at frequencies Fs = 5 MHz.
a) Add Gaussian noise to the sampled sinewave such that the signal SNR is
50 dB. Find first the variance of the Gaussian noise needed to produce the
target SNR. Calculate and plot the Power Spectral Density (PSD) from the DFT of the
noisy samples. Corroborate that the SNR calculation from the DFT plot gives the
theoretical result. What would be the variance of a uniformly distributed noise to obtain
the same SNR.
b) Now repeat a.) using a window before the DFT. Use the following
windows: hann, Hamming, Blackman. What are your conclusions?
Note: The use of windows mentioned above spreads the signal power. You
must take this into account when computing SNR.'''

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hann, hamming, blackman
from scipy.fft import fft, fftfreq

# Part a) Add Gaussian noise and calculate PSD
# Parameters
f_signal = 2e6  # Signal frequency in Hz
A_signal = 1.0  # Signal amplitude in V
Fs = 5e6  # Sampling frequency in Hz
T = 1 / Fs  # Sampling period in seconds
N = 1024  # Number of samples
t = np.arange(N) * T  # Time vector
# Generate the signal

signal = A_signal * np.sin(2 * np.pi * f_signal * t)
# Calculate signal power
P_signal = (A_signal ** 2) / 2
# Desired SNR in dB
SNR_dB = 50
# Calculate noise power from SNR
P_noise = P_signal / (10 ** (SNR_dB / 10))
# Generate Gaussian noise
noise_gaussian = np.random.normal(0, np.sqrt(P_noise), N)
# Noisy signal
noisy_signal_gaussian = signal + noise_gaussian
# Calculate PSD using DFT
fft_gaussian = fft(noisy_signal_gaussian)
freqs = fftfreq(N, T)
PSD_gaussian = (1 / N) * np.abs(fft_gaussian) ** 2
# Plot PSD
plt.figure(figsize=(12, 6))
plt.plot(freqs[:N // 2], 10 * np.log10(PSD_gaussian[:N // 2]))
plt.title('Power Spectral Density of Noisy Signal (Gaussian Noise)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (dB/Hz)')
plt.grid()
plt.xlim(0, Fs / 2)
plt.show()
# Calculate SNR from DFT
signal_power_dft = np.max(PSD_gaussian)
noise_power_dft = np.mean(PSD_gaussian)
SNR_dft = 10 * np.log10(signal_power_dft / noise_power_dft)
print(f"SNR from DFT: {SNR_dft:.2f} dB")
# Calculate variance of uniformly distributed noise for the same SNR
P_noise_uniform = P_signal / (10 ** (SNR_dB / 10))
variance_uniform = P_noise_uniform / 3  # Variance of uniform noise is (b
# - a)^2 / 12, for a uniform distribution between -b and b, variance = b^2 / 3
print(f"Variance of uniformly distributed noise for the same SNR: {variance_uniform:.6e} V^2")


# Part b) Using windows
windows = {
    'hanning': hann(N),
    'Hamming': hamming(N),
    'Blackman': blackman(N)
}
for window_name, window in windows.items():
    # Apply window to the noisy signal
    windowed_signal = noisy_signal_gaussian * window
    # Calculate PSD using DFT
    fft_windowed = fft(windowed_signal)
    PSD_windowed = (1 / N) * np.abs(fft_windowed) ** 2
    # Plot PSD
    plt.figure(figsize=(12, 6))
    plt.plot(freqs[:N // 2], 10 * np.log10(PSD_windowed[:N // 2]))
    plt.title(f'Power Spectral Density of Noisy Signal with {window_name} Window')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD (dB/Hz)')
    plt.grid()
    plt.xlim(0, Fs / 2)
    plt.show()
    # Calculate SNR from DFT
    signal_power_dft_windowed = np.max(PSD_windowed)
    noise_power_dft_windowed = np.mean(PSD_windowed)
    SNR_dft_windowed = 10 * np.log10(signal_power_dft_windowed / noise_power_dft_windowed)
    print(f"SNR from DFT with {window_name} window: {SNR_dft_windowed:.2f} dB")


ImportError: cannot import name 'hanning' from 'scipy.signal' (c:\Users\Lmaoi\anaconda3\envs\huggingface_env\lib\site-packages\scipy\signal\__init__.py)

**Part c: Conclusions:**
The use of windows affects the spectral leakage and the distribution of signal power in the frequency domain. The SNR calculated from the DFT may vary depending on the window used, as windows can spread the signal power across multiple frequency bins, which can affect the noise power estimation.


In [ ]:
'''2. QUANTIZATION (60)
a) Create a perfect quantizer with 6 bits of resolution and flexible sampling rate. For a
200 MHz full scale input tone, sample and quantize the sinewave at 400 MHz and plot
the PSD of 30 periods. What is the SNR? Repeat the SNR calculation for 100 periods of
the same signal. Make your own conclusions about this test regarding periodicity of
quantization noise and the impact of this in the SNR. How can you solve this problem?
b) Find an incommensurate sampling frequency larger than Nyquist rate.
Plot the PSD of the new samples. Calculate the SNR from the figure.
c) Repeat a) using a 12 bit quantizer. Can you prove from simulations that
SNR ~ 6N (where N is the number of bits used by the quantizer) in both the
cases, N = 6 and N = 12?
d) Use a Hanning window and repeat c). What is the SNR? Make your own
conclusions.
e) Now add noise again so the signal SNR is 38 dB . Repeat c) and d). What
are the SNRs? Provide conclusions.
HINT: Write a python function using the numpy.round to perform Quantization on your
wavform.'''

#Part a) Create a perfect quantizer with 6 bits of resolution
def quantize(signal, bits):
    levels = 2 ** bits
    quantized_signal = np.round((signal + A_signal) / (2 * A_signal) * (levels - 1)) / (levels - 1) * (2 * A_signal) - A_signal
    return quantized_signal
# Parameters
f_signal = 200e6  # Signal frequency in Hz
A_signal = 1.0  # Signal amplitude in V
Fs = 400e6  # Sampling frequency in Hz
T = 1 / Fs  # Sampling period in seconds
N = 1024  # Number of samples for 30 periods
t = np.arange(N) * T  # Time vector
# Generate the signal
signal = A_signal * np.sin(2 * np.pi * f_signal * t)
# Quantize the signal with 6 bits
quantized_signal_6bit = quantize(signal, 6)
# Calculate PSD using DFT
fft_quantized_6bit = fft(quantized_signal_6bit)
freqs = fftfreq(N, T)
PSD_quantized_6bit = (1 / N) * np.abs(fft_quantized_6bit) ** 2
# Plot PSD
plt.figure(figsize=(12, 6))
plt.plot(freqs[:N // 2], 10 * np.log10(PSD_quantized_6bit[:N // 2]))
plt.title('Power Spectral Density of Quantized Signal (6 bits)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (dB/Hz)')
plt.grid()
plt.xlim(0, Fs / 2)
plt.show()
# Calculate SNR from DFT
signal_power_dft_6bit = np.max(PSD_quantized_6bit)
noise_power_dft_6bit = np.mean(PSD_quantized_6bit)
SNR_dft_6bit = 10 * np.log10(signal_power_dft_6bit / noise_power_dft_6bit)
print(f"SNR from DFT with 6-bit quantizer: {SNR_dft_6bit:.2f} dB")
# Repeat for 100 periods
N_100 = 1024 * 100  # Number of samples for 100 periods
t_100 = np.arange(N_100) * T  # Time vector for 100
signal_100 = A_signal * np.sin(2 * np.pi * f_signal * t_100)
quantized_signal_6bit_100 = quantize(signal_100, 6)
fft_quantized_6bit_100 = fft(quantized_signal_6bit_100)
freqs_100 = fftfreq(N_100, T)
PSD_quantized_6bit_100 = (1 / N_100) * np.abs
(fft_quantized_6bit_100) ** 2
# Plot PSD for 100 periods
plt.figure(figsize=(12, 6))
plt.plot(freqs_100[:N_100 // 2], 10 * np.log10(PSD_quantized_6bit_100[:N_100 // 2]))
plt.title('Power Spectral Density of Quantized Signal (6 bits, 100 periods)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (dB/Hz)')
plt.grid()
plt.xlim(0, Fs / 2)
plt.show()
# Calculate SNR from DFT for 100 periods
signal_power_dft_6bit_100 = np.max(PSD_quantized_6bit_100)
noise_power_dft_6bit_100 = np.mean(PSD_quantized_6bit_100)
SNR_dft_6bit_100 = 10 * np.log10(signal_power_dft_6bit_100 / noise_power_dft_6bit_100)
print(f"SNR from DFT with 6-bit quantizer (100 periods): {SNR_dft_6bit_100:.2f} dB")

#Part b) Find an incommensurate sampling frequency larger than Nyquist rate
# Nyquist rate is 2 * f_signal = 400 MHz, so we can choose a sampling frequency that is not an integer multiple of the signal frequency, for example:
Fs_incommensurate = 450e6  # Incommensurate sampling frequency in Hz
T_incommensurate = 1 / Fs_incommensurate  # Sampling period in seconds
N_incommensurate = 1024  # Number of samples
t_incommensurate = np.arange(N_incommensurate) * T_incommensurate  # Time vector
# Generate the signal
signal_incommensurate = A_signal * np.sin(2 * np.pi * f_signal * t_incommensurate)
# Quantize the signal with 6 bits
quantized_signal_incommensurate = quantize(signal_incommensurate, 6)
# Calculate PSD using DFT
fft_quantized_incommensurate = fft(quantized_signal_incommensurate)
freqs_incommensurate = fftfreq(N_incommensurate, T_incommensurate)
PSD_quantized_incommensurate = (1 / N_incommensurate) * np.abs(fft_quantized_incommensurate) ** 2
# Plot PSD
plt.figure(figsize=(12, 6))
plt.plot(freqs_incommensurate[:N_incommensurate // 2], 10 * np.log10(PSD_quantized_incommensurate[:N_incommensurate // 2]))
plt.title('Power Spectral Density of Quantized Signal (6 bits, Incommensurate Sampling)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (dB/Hz)')
plt.grid()
plt.xlim(0, Fs_incommensurate / 2)
plt.show()
# Calculate SNR from DFT
signal_power_dft_incommensurate = np.max(PSD_quantized_incommensurate)
noise_power_dft_incommensurate = np.mean(PSD_quantized_incommensurate)
SNR_dft_incommensurate = 10 * np.log10(signal_power_dft_incommensurate / noise_power_dft_incommensurate)
print(f"SNR from DFT with 6-bit quantizer (incommensurate sampling): {SNR_dft_incommensurate:.2f} dB")

#Part c) Repeat a) using a 12 bit quantizer
# Quantize the signal with 12 bits
quantized_signal_12bit = quantize(signal, 12)
# Calculate PSD using DFT
fft_quantized_12bit = fft(quantized_signal_12bit)
PSD_quantized_12bit = (1 / N) * np.abs(fft_quantized_12bit) ** 2
# Plot PSD
plt.figure(figsize=(12, 6))
plt.plot(freqs[:N // 2], 10 * np.log10(PSD_quantized_12bit[:N // 2]))
plt.title('Power Spectral Density of Quantized Signal (12 bits)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (dB/Hz)')
plt.grid()
plt.xlim(0, Fs / 2)
plt.show()
# Calculate SNR from DFT
signal_power_dft_12bit = np.max(PSD_quantized_12bit)
noise_power_dft_12bit = np.mean(PSD_quantized_12bit)
SNR_dft_12bit = 10 * np.log10(signal_power_dft_12bit / noise_power_dft_12bit)
print(f"SNR from DFT with 12-bit quantizer: {SNR_dft_12bit:.2f} dB")
theoretical_SNR_6bit = 6.02 * 6 + 1.76
theoretical_SNR_12bit = 6.02 * 12 + 1.76
print(f"Theoretical SNR for 6-bit quantizer: {theoretical_SNR_6bit:.2f} dB")
print(f"Theoretical SNR for 12-bit quantizer: {theoretical_SNR_12bit:.2f} dB")


# Part d) Use a Hanning window and repeat c) 
window_hanning = hann(N)
windowed_signal_12bit = quantized_signal_12bit * window_hanning
fft_windowed_12bit = fft(windowed_signal_12bit)
PSD_windowed_12bit = (1 / N) * np.abs(fft_windowed_12bit) ** 2 
# Plot PSD
plt.figure(figsize=(12, 6))
plt.plot(freqs[:N // 2], 10 * np.log10(PSD_windowed_12bit[:N // 2]))
plt.title('Power Spectral Density of Quantized Signal (12 bits, Hanning Window)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (dB/Hz)')
plt.grid()
plt.xlim(0, Fs / 2)
plt.show()
# Calculate SNR from DFT
signal_power_dft_windowed_12bit = np.max(PSD_windowed_12bit)
noise_power_dft_windowed_12bit = np.mean(PSD_windowed_12bit)
SNR_dft_windowed_12bit = 10 * np.log10(signal_power_dft_windowed_12bit / noise_power_dft_windowed_12bit)
print(f"SNR from DFT with 12-bit quantizer and Hanning window: {SNR_dft_windowed_12bit:.2f} dB")

#Part e) Now add noise again so the signal SNR is 38 dB . Repeat c) and d)
# Desired SNR in dB
SNR_dB_noisy = 38
# Calculate noise power from SNR
P_noise_noisy = P_signal / (10 ** (SNR_dB_noisy / 10))
# Generate Gaussian noise
noise_gaussian_noisy = np.random.normal(0, np.sqrt(P_noise_noisy), N)
# Noisy signal
noisy_signal_12bit = quantized_signal_12bit + noise_gaussian_noisy
# Calculate PSD using DFT
fft_noisy_12bit = fft(noisy_signal_12bit)
PSD_noisy_12bit = (1 / N) * np.abs(fft_noisy_12bit) ** 2
# Plot PSD
plt.figure(figsize=(12, 6))
plt.plot(freqs[:N // 2], 10 * np.log10(PSD_noisy_12bit[:N // 2]))
plt.title('Power Spectral Density of Noisy Quantized Signal (12 bits)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (dB/Hz)')
plt.grid()
plt.xlim(0, Fs / 2)
plt.show()
# Calculate SNR from DFT
signal_power_dft_noisy_12bit = np.max(PSD_noisy_12bit)
noise_power_dft_noisy_12bit = np.mean(PSD_noisy_12bit)
SNR_dft_noisy_12bit = 10 * np.log10(signal_power_dft_noisy_12bit / noise_power_dft_noisy_12bit)
print(f"SNR from DFT with 12-bit quantizer and noise: {SNR_dft_noisy_12bit:.2f} dB")
# Now apply Hanning window to the noisy signal
windowed_noisy_signal_12bit = noisy_signal_12bit * window_hanning
fft_windowed_noisy_12bit = fft(windowed_noisy_signal_12bit)
PSD_windowed_noisy_12bit = (1 / N) * np.abs(fft_windowed_noisy_12bit) ** 2
# Plot PSD
plt.figure(figsize=(12, 6))
plt.plot(freqs[:N // 2], 10 * np.log10(PSD_windowed_noisy_12bit[:N // 2]))
plt.title('Power Spectral Density of Noisy Quantized Signal (12 bits, Hanning Window)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (dB/Hz)')
plt.grid()
plt.xlim(0, Fs / 2)
plt.show()
# Calculate SNR from DFT
signal_power_dft_windowed_noisy_12bit = np.max(PSD_windowed_noisy_12bit)
noise_power_dft_windowed_noisy_12bit = np.mean(PSD_windowed_noisy_12bit)
SNR_dft_windowed_noisy_12bit = 10 * np.log10(signal_power_dft_windowed_noisy_12bit / noise_power_dft_windowed_noisy_12bit)
print(f"SNR from DFT with 12-bit quantizer, noise, and Hanning window: {SNR_dft_windowed_noisy_12bit:.2f} dB")

                                    


**Part a**/
The SNR calculated from the DFT can vary depending on the number of periods analyzed, as the quantization noise can have periodic components that may affect the noise power estimation. To solve this problem, one can use a larger number of samples (more periods) to average out the effects of periodic quantization noise, or apply a window to reduce spectral leakage.

**Part b**/
The SNR calculated from the DFT with incommensurate sampling may differ from the commensurate case due to the different distribution of quantization noise in the frequency domain, which can affect the noise power estimation.

**Part c**/
Theoretical SNR for quantization noise is given by SNR = 6.02 * N + 1.76 dB, where N is the number of bits.
The SNR from the DFT should be close to the theoretical SNR for both cases, confirming that SNR ~ 6N for quantization noise.

**Part d**/
The use of a Hanning window can affect the SNR calculated from the DFT, as it can spread the signal power across multiple frequency bins, which can affect the noise power estimation. The SNR may be lower than the theoretical value due to the windowing effect on the signal and noise distribution in the frequency domain.

**Part e**/ 
The addition of noise significantly reduces the SNR calculated from the DFT, as expected. The use of a Hanning window in the presence of noise can further affect the SNR estimation due to the spreading of signal power and noise in the frequency domain, which can lead to a lower SNR than the theoretical value.        



